In [1]:
from osgeo import ogr 
import os 
ogr.UseExceptions()

In [28]:
basedir = "/media/riannek/minimax/gleis"
tmpdir = "/media/riannek/minimax/gleis/temp_fertig"
gpkg = "lasfiles_run14_und_run24.gpkg"
layer_name = "lasfile"

In [34]:
gpkg = ogr.Open(os.path.join(tmpdir, "temp.gpkg"))

layer = gpkg.GetLayerByName("tiles")

In [7]:
def extend_bbox(bbox, margin=2):
    # MinX, MaxX, MinY, MaxY
    return (bbox[0] - margin, bbox[1] + margin, bbox[2] - margin, bbox[3] + margin)

In [8]:
def get_bbox_polygon(bbox):  
    ring = ogr.Geometry(ogr.wkbLinearRing)      
    ring.AddPoint_2D(bbox[0], bbox[2])  # MinX, MinY
    ring.AddPoint_2D(bbox[1], bbox[2])  # MaxX, MinY
    ring.AddPoint_2D(bbox[1], bbox[3])  # MaxX, MaxY
    ring.AddPoint_2D(bbox[0], bbox[3])  # MinX, MaxY
    ring.AddPoint_2D(bbox[0], bbox[2])  # Close ring
    geom = ogr.Geometry(ogr.wkbPolygon)
    geom.AddGeometry(ring)
    return geom

In [37]:

def tile_generator(layer):
    bounding_boxes = []
    for feature in layer:
        bbox = feature.GetGeometryRef().GetEnvelope()
        bounding_boxes.append(bbox)

    for bbox in bounding_boxes:
        extended = extend_bbox(bbox)
        layer.ResetReading()
        

        bbox_geom = get_bbox_polygon(extended)
        layer.SetSpatialFilter(bbox_geom)

        tiles = [f.GetField("filename") for f in layer]
        
        layer.SetSpatialFilter(None)
        yield bbox, extended, tiles


In [ ]:
#layer.ResetReading()

In [38]:
generator = tile_generator(layer)

In [39]:
bbox, extend_bbox, tiles = next(generator)
print(bbox) 
print(extend_bbox) 
tiles

(4475525.0, 4475550.0, 5341275.0, 5341300.0)
(4475523.0, 4475552.0, 5341273.0, 5341302.0)


['4475525_5341275.ply',
 '4475500_5341250.ply',
 '4475525_5341300.ply',
 '4475550_5341250.ply',
 '4475550_5341300.ply',
 '4475525_5341250.ply',
 '4475500_5341275.ply',
 '4475500_5341300.ply',
 '4475550_5341275.ply']

In [270]:
get_bbox_polygon(bbox).ExportToWkt()

'POLYGON ((4474108.522 4474125.0,5333500 4474125,5333500.0 5333518.616,4474108.522 5333518.616,4474108.522 4474125.0))'